**Ingestão dos dados com Python via URL**

In [1]:
import requests

In [2]:
url = 'https://dados.anvisa.gov.br/dados/TA_PRECOS_MEDICAMENTOS.csv'
url2 = 'https://dados.anvisa.gov.br/dados/TA_PRECO_MEDICAMENTO_GOV.CSV'

In [3]:
r = requests.get(url, verify=False)

/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dados.anvisa.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [4]:
l = requests.get(url2, verify=False)

/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dados.anvisa.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [5]:
with open ("r.csv", "wb") as f:
    f.write(r.content)

In [6]:
with open ("l.csv", "wb") as g:
    g.write(l.content)

**Tratando os dados através do Spark, criando os Data Frames e transformando os nomes das colunas para letras minúsculas**

In [7]:
from pyspark.sql import SparkSession

In [8]:
spark = SparkSession \
.builder \
.config("spark.sql.repl.eagerEval.enabled", "True") \
.config("spark.sql.repl.eagerEval.maxNumRows", "10") \
.getOrCreate()

In [9]:
import pyspark.sql.functions as f

In [10]:
df1 = spark.read.csv("r.csv", header=True, sep=";", inferSchema=True)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
from pyspark.sql.functions import col
df1 = df1.select([col(c).alias(c.lower()) for c in df1.columns])

In [ ]:
df1.createOrReplaceTempView('dados2023')

In [ ]:
df2 = spark.read.csv("l.csv", header=True, sep=";", inferSchema=True)

In [ ]:
df2 = df2.select([col(c).alias(c.lower()) for c in df2.columns])

In [ ]:
df2.createOrReplaceTempView('dados2022')

In [ ]:
df2 = df2.withColumnRenamed("princ�pio ativo","principio_ativo") \
.withColumnRenamed("laborat�rio","laboratorio") \
.withColumnRenamed("c�digo ggrem","codigo") \
.withColumnRenamed("apresenta��o", "apresentacao") \
.withColumnRenamed("classe terap�utica","classe_teraupetica") \
.withColumnRenamed("regime de pre�o","regime_preco") \
.withColumnRenamed("restri��o hospitalar","restricao_hospitalar") \
.withColumnRenamed("an�lise recursal","analise_recursal") \
.withColumnRenamed("lista de concess�o de cr�dito tribut�rio (pis/cofins)","pis_cofins") \
.withColumnRenamed("comercializa��o 2022","comercializacao_2022")

**Pergunta número 1 - Quais os laboratórios com preço de Fábrica da substância Dipirona mais cara e mais barata no Brasil ?**

In [ ]:
spark.sql("""
  SELECT no_razao_social AS Laboratorio, MAX(nu_pf18_inteiro) AS PrecoMaisCaro, MIN(nu_pf18_inteiro) AS PrecoMaisBarato
  FROM dados2023
  WHERE no_produto = 'DIPIRONA'
  AND nu_pf18_inteiro > '1'
  GROUP BY no_razao_social
  """)


**Pergunta Número 2 - Quantos laboratórios diferentes possuem registros para a substância Clonazepam e qual destes possui o menor preço ?**

In [ ]:
spark.sql("""
    SELECT COUNT(DISTINCT no_razao_social) AS Quantidade_Laboratorios_Com_Clonazepam
    FROM dados2023
    WHERE ds_substancia = 'clonazepam'
""")

In [ ]:
spark.sql("""
    SELECT no_razao_social AS Laboratorio_mais_em_conta
    FROM dados2023
    WHERE ds_substancia = 'clonazepam'
    GROUP BY no_razao_social
    LIMIT 1
""")



**Pergunta Número 3 - Quais as substâncias com menores preços que têm vendas restritas apenas para hospitais?**

In [ ]:
spark.sql("""
SELECT no_produto AS Produto, ds_substancia AS Substancia, nu_pf18_inteiro AS Preco, st_rest_hosp AS RestricaoHospitalar
FROM dados2023
WHERE st_rest_hosp = 'S'
AND nu_pf18_inteiro = (
            SELECT MIN(nu_pf18_inteiro)
            FROM dados2023
                WHERE nu_pf18_inteiro > '1')
          """)

**Pergunta Número 4 - Qual o nome do produto e o laboratório do preço de fábrica mais baixo de sulfato de morfina no Brasil ?**

In [ ]:
spark.sql("""
SELECT no_razao_social Laboratorio, no_produto Nome_Produto, nu_pf18_inteiro AS Preco
    FROM dados2023
    WHERE ds_substancia = 'sulfato de morfina'
        AND nu_pf18_inteiro > '1'
        AND nu_pf18_inteiro = (
            SELECT MIN(nu_pf18_inteiro)
            FROM dados2023
            WHERE ds_substancia = 'sulfato de morfina'
                AND nu_pf18_inteiro > '1'
        )
        """)


**Pergunta Número 5 - Qual o laboratório de medicamentos com o menor preço de fábrica com ICMS de 18% no Brasil ?**

In [ ]:
spark.sql("""
SELECT DISTINCT(no_razao_social) Laboratorios_com_menores_precos, no_produto AS Produto, nu_pf18_inteiro AS MenorPrecoDeFabrica_ICMS_18
FROM dados2023
WHERE nu_pf18_inteiro = (
    SELECT MIN(nu_pf18_inteiro) Menor_preco
    FROM dados2023
    WHERE nu_pf18_inteiro > '1'
        )""")

**Realizando o Output dos DataFrames em .Parquet**

In [ ]:
df1.write.parquet("work/data/dados2023.parquet")

In [ ]:
df2.write.parquet("work/data/dados2022.parquet")